<a href="https://colab.research.google.com/drive/1djItMLuXd7prvVnfqTEpNDYV27CE7X4G?usp=sharing" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os
import json
#sur collab
#dossier Transformer.zip a import + dataset.json a importer
#mode exec GPU

In [2]:
!unzip Transformer.zip

Archive:  Transformer.zip
 extracting: Transformer/__init__.py  
  inflating: Transformer/melodygenerator.py  
  inflating: Transformer/melodypreprocessor.py  
  inflating: Transformer/train.py    
  inflating: Transformer/transformer.py  


In [3]:
sys.path.append('/content/Transformer')

In [4]:
from melodygenerator import *
from melodypreprocessor import *
from train import *
from transformer import *
from train import _train_step

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
melody_preprocessor = MelodyPreprocessor("dataset.json", batch_size=32)
train_dataset = melody_preprocessor.create_training_dataset()
vocab_size = melody_preprocessor.number_of_tokens_with_padding
transformer_model = Transformer(
    num_layers=2,
    d_model=64,
    num_heads=2,
    d_feedforward=128,
    input_vocab_size=vocab_size,
    target_vocab_size=vocab_size,
    max_num_positions_in_pe_encoder=MAX_POSITIONS_IN_POSITIONAL_ENCODING,
    max_num_positions_in_pe_decoder=MAX_POSITIONS_IN_POSITIONAL_ENCODING,
    dropout_rate=0.1,
)

with tf.device('/device:GPU:0'):
    train(train_dataset, transformer_model, 10)



Training the model...
Epoch 1 Batch 1 Loss 4.886707305908203
Epoch 1 Batch 2 Loss 3.927978038787842
Epoch 1 Batch 3 Loss 3.748643398284912
Epoch 1 Batch 4 Loss 3.2022767066955566
Epoch 1 Batch 5 Loss 3.3923604488372803
Epoch 1 Batch 6 Loss 3.477008104324341
Epoch 1 Batch 7 Loss 3.3242170810699463
Epoch 1 Batch 8 Loss 3.373722791671753
Epoch 1 Batch 9 Loss 3.5260207653045654
Epoch 1 Batch 10 Loss 3.553171157836914
Epoch 1 Batch 11 Loss 3.471177339553833
Epoch 1 Batch 12 Loss 3.4750421047210693
Epoch 1 Batch 13 Loss 3.654526472091675
Epoch 1 Batch 14 Loss 3.8982021808624268
Epoch 1 Batch 15 Loss 3.6468071937561035
Epoch 1 Batch 16 Loss 3.4628090858459473
Epoch 1 Batch 17 Loss 3.5853657722473145
Epoch 1 Batch 18 Loss 3.3916122913360596
Epoch 1 Batch 19 Loss 3.3624322414398193
Epoch 1 Batch 20 Loss 3.199434280395508
Epoch 1 Batch 21 Loss 3.2610533237457275
Epoch 1 Batch 22 Loss 3.280744791030884
Epoch 1 Batch 23 Loss 3.0921335220336914
Epoch 1 Batch 24 Loss 2.965222120285034
Epoch 1 Batch 

In [6]:
melody_generator = MelodyGenerator(
    transformer_model, melody_preprocessor.tokenizer,100
)

In [7]:
import random
import json

with open("dataset.json","r") as f:
    melodies= json.load(f)

In [8]:
k=4 # longueur de la sequence debut
print("Generating a melody...")
start_sequence = random.choice(melodies).split()[:k]
start_sequence = [ s[:-1] for s in start_sequence]
print(start_sequence)
new_melody = melody_generator.generate(start_sequence)
print(new_melody)

Generating a melody...
['D4-1.0', 'E4-1.0']
D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0


In [ ]:
melodies_generated=[]

In [12]:
k=4
for _ in range(10):
  print("Generating a melody...")
  start_sequence = random.choice(melodies).split()[:k]
  start_sequence = [ s[:-1] for s in start_sequence]
  print(start_sequence)
  new_melody = melody_generator.generate(start_sequence)

  melodies_generated.append((start_sequence,new_melody))
for i,(s,m) in enumerate(melodies_generated,1):
    print(i,m)

Generating a melody...
['E5-1.0', 'B4-0.25', 'D5-0.25', 'D-5-0.25']
Generating a melody...
['E5-0.25', 'D5-0.25', 'C5-0.25', 'B4-0.25']
Generating a melody...
['D6-0.25', 'D-6-0.25', 'C6-0.25', 'C5-0.25']
Generating a melody...
['C#6-0.25', 'C6-0.25', 'B5-0.25', 'A5-0.25']
Generating a melody...
['G4-0.5', 'A4-0.25', 'C5-0.25', 'F4-0.25']
Generating a melody...
['C4-0.5', 'F4-0.5', 'E4-0.5', 'C4-0.5']
Generating a melody...
['E5-0.5', 'E5-0.25', 'E5-0.25', 'F5-0.5']
Generating a melody...
['E5-0.25', 'D5-0.25', 'C5-0.25', 'B4-0.25']
Generating a melody...
['A4-0.25', 'B4-0.25', 'C5-0.25', 'E5-0.25']
Generating a melody...
['D4-2.0', 'D4-2.0', 'D4-2.0', 'D4-2.0']
1 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 E6-0.25 E6-0.25 E6-0.25 G5-0.25 G5-0.25 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 

In [13]:
generated = [ {'debut' : s , 'melody' : m } for (s,m) in melodies_generated ]
print(generated)

[{'debut': ['D5-2.0', 'D5-2.0'], 'melody': 'D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 D5-2.0 E6-0.25 E6-0.25 E6-0.25 G5-0.25 G5-0.25 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 D4-2.0 B-4-0.25 G4-0.25 C5-0.5 E4-0.25 G4-0.25 G4-0.25 E4-0.5 A4-0.25 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0 F4-1.0 D4-1.0 E4-1.0'}, {'debut': ['C4-0.5', 'F4-0.5'], 'melody': 'C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4-0.5 E4-0.5 C4-0.5 F4

In [15]:
with open("generated.json","w") as f:
    json.dump(generated,f,indent=2)